<a href="https://colab.research.google.com/github/Ratnesh-bhosale/Blockchain_Using_Python/blob/main/Blockchain_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import json
from time import time
from urllib.parse import urlparse
from uuid import uuid4
import requests
from flask import Flask, jsonify, request

In [ ]:
class Blockchain(object):
  def __init__(self):
    self.current_transactions = []
    self.chain = []
    self.nodes = set()

    # Create the genesis block
    self.new_block(previous_hash=1, proof=100)
  
  def register_node(self, address):
    """
    Add a new node to the list of nodes

    :param address: <str> Address of node. Eg. 'http://192.168.0.5:5000'
    :return: None
    """

    parsed_url = urlparse(address)
    self.nodes.add(parsed_url.netloc)

  def valid_chain(self, chain):
    """
    Determine if a given blockchain is valid

    :param chain: <list> A blockchain
    :return: <bool> True if valid, False if not
    """

    last_block = chain[0]
    current_index = 1

    while current_index < len(chain):
      block = chain[current_index]
      print(f'{last_block}')
      print(f'{block}')
      print("\n-----------\n")
      # Check that the hash of the block is correct
      if block['previous_hash'] != self.hash(last_block):
        return False

      # Check that the Proof of Work is correct
      if not self.valid_proof(last_block['proof'], block['proof']):
        return False

      last_block = block
      current_index += 1

    return True
  
  def resolve_conflicts(self):
    """
    This is our consensus algorithm, it resolves conflicts
    by replacing our chain with the longest one in the network.

    :return: <bool> True if our chain was replaced, False if not
    """

    neighbours = self.nodes
    new_chain = None

    # We're only looking for chains longer than ours
    max_length = len(self.chain)

    # Grab and verify the chains from all the nodes in our network
    for node in neighbours:
      response = requests.get(f'http://{node}/chain')

      if response.status_code == 200:
        length = response.json()['length']
        chain = response.json()['chain']

        # Check if the length is longer and the chain is valid
        if length > max_length and self.valid_chain(chain):
          max_length = length
          new_chain = chain

    # Replace our chain if we discovered a new, valid chain longer than ours
    if new_chain:
      self.chain = new_chain
      return True

    return False

  def new_block(self, proof, previous_hash=None):
    """
    Create a new Block in the Blockchain

    :param proof: <int> The proof given by the Proof of Work algorithm
    :param previous_hash: (Optional) <str> Hash of previous Block
    :return: <dict> New Block
    """

    block = {
      'index': len(self.chain) + 1,
      'timestamp': time(),
      'transactions': self.current_transactions,
      'proof': proof,
      'previous_hash': previous_hash or self.hash(self.chain[-1]),
    }

    # Reset the current list of transactions
    self.current_transactions = []

    self.chain.append(block)
    return block

  def new_transaction(self, sender, recipient, amount):
    """
    Creates a new transaction to go into the next mined Block

    :param sender: <str> Address of the Sender
    :param recipient: <str> Address of the Recipient
    :param amount: <int> Amount
    :return: <int> The index of the Block that will hold this transaction
    """
    self.current_transactions.append({
      'sender': sender,
      'recipient': recipient,
      'amount': amount,
    })

    return self.last_block['index'] + 1

  @property
  def last_block(self):
    return self.chain[-1]
  
  @staticmethod
  def hash(block):
    """
    Creates a SHA-256 hash of a Block

    :param block: <dict> Block
    :return: <str>
    """

    # We must make sure that the Dictionary is Ordered, or we'll have inconsistent hashes
    block_string = json.dumps(block, sort_keys=True).encode()
    return hashlib.sha256(block_string).hexdigest()

  def proof_of_work(self, last_proof):
    """
    Simple Proof of Work Algorithm:
      - Find a number p' such that hash(pp') contains leading 4 zeroes, where p is the previous p'
      - p is the previous proof, and p' is the new proof

    :param last_proof: <int>
    :return: <int>
    """

    proof = 0
    while self.valid_proof(last_proof, proof) is False:
      proof += 1

    return proof

  @staticmethod
  def valid_proof(last_proof, proof):
    """
    Validates the Proof

    :param last_proof: <int> Previous Proof
    :param proof: <int> Current Proof
    :return: <bool> True if correct, False if not.
    """

    guess = f'{last_proof}{proof}'.encode()
    guess_hash = hashlib.sha256(guess).hexdigest()
    return guess_hash[:4] == "0000"

In [ ]:
def create_blockchain_process(in_port):
    # Generate a globally unique address for this node
    node_identifier = '%s%04d' % (str(uuid4()).replace('-', ''), in_port)
    # Instantiate the Node
    app = Flask(node_identifier)
    # Instantiate the Blockchain
    blockchain = Blockchain()
    @app.route('/mine', methods=['GET'])
    def mine():
        # We run the proof of work algorithm to get the next proof...
        last_block = blockchain.last_block
        last_proof = last_block['proof']
        proof = blockchain.proof_of_work(last_proof)

        # We must receive a reward for finding the proof.
        # The sender is "0" to signify that this node has mined a new coin.
        blockchain.new_transaction(
            sender="0",
            recipient=node_identifier,
            amount=1,
        )

        # Forge the new Block by adding it to the chain
        block = blockchain.new_block(proof)

        response = {
            'message': "New Block Forged",
            'index': block['index'],
            'transactions': block['transactions'],
            'proof': block['proof'],
            'previous_hash': block['previous_hash'],
        }
        return jsonify(response), 200


    @app.route('/transactions/new', methods=['POST'])
    def new_transaction():
        values = request.get_json()

        # Check that the required fields are in the POST'ed data
        required = ['sender', 'recipient', 'amount']
        if not all(k in values for k in required):
            return 'Missing values', 400

        # Create a new Transaction
        index = blockchain.new_transaction(values['sender'], values['recipient'], values['amount'])

        response = {'message': f'Transaction will be added to Block {index}'}
        return jsonify(response), 201


    @app.route('/chain', methods=['GET'])
    def full_chain():
        response = {
            'chain': blockchain.chain,
            'length': len(blockchain.chain),
        }
        return jsonify(response), 200


    @app.route('/nodes/register', methods=['POST'])
    def register_nodes():
        values = request.get_json()

        nodes = values.get('nodes')
        if nodes is None:
            return "Error: Please supply a valid list of nodes", 400

        for node in nodes:
            blockchain.register_node(node)

        response = {
            'message': 'New nodes have been added',
            'total_nodes': list(blockchain.nodes),
        }
        return jsonify(response), 201


    @app.route('/nodes/resolve', methods=['GET'])
    def consensus():
        replaced = blockchain.resolve_conflicts()

        if replaced:
            response = {
                'message': 'Our chain was replaced',
                'new_chain': blockchain.chain
            }
        else:
            response = {
                'message': 'Our chain is authoritative',
                'chain': blockchain.chain
            }

        return jsonify(response), 200
    app.run(host='0.0.0.0', port=in_port)
    os._exit(0)

In [ ]:
import os
from time import sleep
out_pids = []
for i in range(5000, 5003):
    cpid = os.fork()
    if not cpid:
        create_blockchain_process(i)
    else:
        out_pids += [cpid]
sleep(2) # make sure everything is started

 * Serving Flask app "3f0c3386a2684118ad9516a858bc8d6d5000" (lazy loading)
 * Environment: production
 * Serving Flask app "e38d3bd541244d3c9be583550af542c05002" (lazy loading)
 * Environment: production
 * Serving Flask app "a5afc77a149047c384f899f2b84bfbb05001" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Debug mode: off
   Use a production WSGI server instead.


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


   Use a production WSGI server instead.


 * Running on http://0.0.0.0:5001/ (Press CTRL+C to quit)


 * Debug mode: off


 * Running on http://0.0.0.0:5002/ (Press CTRL+C to quit)


In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '{"nodes": ["http://localhost:5001", "http://localhost:5002"]}' "http://localhost:5000/nodes/register"
!curl -X POST -H "Content-Type: application/json" -d '{"nodes": ["http://localhost:5000", "http://localhost:5002"]}' "http://localhost:5001/nodes/register"
!curl -X POST -H "Content-Type: application/json" -d '{"nodes": ["http://localhost:5000", "http://localhost:5001"]}' "http://localhost:5002/nodes/register"

{"message":"New nodes have been added","total_nodes":["localhost:5001","localhost:5002"]}
{"message":"New nodes have been added","total_nodes":["localhost:5000","localhost:5002"]}
{"message":"New nodes have been added","total_nodes":["localhost:5000","localhost:5001"]}


In [ ]:
!curl -X POST -H "Content-Type: application/json" -d '{ "sender": "d4ee26eee15148ee92c6cd394edd974e","recipient": "someone-other-address", "amount": 5}' "http://localhost:5000/transactions/new"

{"message":"Transaction will be added to Block 2"}


In [ ]:
!curl http://localhost:5000/chain

{"chain":[{"index":1,"previous_hash":1,"proof":100,"timestamp":1640294608.456102,"transactions":[]}],"length":1}


In [ ]:
!curl http://localhost:5000/nodes/resolve
!curl http://localhost:5001/nodes/resolve
!curl http://localhost:5002/nodes/resolve

{"chain":[{"index":1,"previous_hash":1,"proof":100,"timestamp":1640294608.456102,"transactions":[]}],"message":"Our chain is authoritative"}
{"chain":[{"index":1,"previous_hash":1,"proof":100,"timestamp":1640294608.4698675,"transactions":[]}],"message":"Our chain is authoritative"}
{"chain":[{"index":1,"previous_hash":1,"proof":100,"timestamp":1640294608.4728572,"transactions":[]}],"message":"Our chain is authoritative"}


In [ ]:
!curl http://localhost:5000/mine

{"index":2,"message":"New Block Forged","previous_hash":"a3cdfeb5efdf64ac0844e10ff0ddf7fff53bb1283bf2bacaa187cb75a8805869","proof":35293,"transactions":[{"amount":5,"recipient":"someone-other-address","sender":"d4ee26eee15148ee92c6cd394edd974e"},{"amount":1,"recipient":"3f0c3386a2684118ad9516a858bc8d6d5000","sender":"0"}]}


In [ ]:
!curl http://localhost:5000/chain

{"chain":[{"index":1,"previous_hash":1,"proof":100,"timestamp":1640294608.456102,"transactions":[]},{"index":2,"previous_hash":"a3cdfeb5efdf64ac0844e10ff0ddf7fff53bb1283bf2bacaa187cb75a8805869","proof":35293,"timestamp":1640294690.6846914,"transactions":[{"amount":5,"recipient":"someone-other-address","sender":"d4ee26eee15148ee92c6cd394edd974e"},{"amount":1,"recipient":"3f0c3386a2684118ad9516a858bc8d6d5000","sender":"0"}]}],"length":2}


In [ ]:
!curl http://localhost:5001/mine
!curl http://localhost:5000/nodes/resolve
!curl http://localhost:5001/nodes/resolve
!curl http://localhost:5002/nodes/resolve

{"index":2,"message":"New Block Forged","previous_hash":"29a0086c3fc5334e1e58dc8b801ac9f01ccd998e900cb01a0faf05be2bf8dae0","proof":35293,"transactions":[{"amount":1,"recipient":"a5afc77a149047c384f899f2b84bfbb05001","sender":"0"}]}
{"chain":[{"index":1,"previous_hash":1,"proof":100,"timestamp":1640294608.456102,"transactions":[]},{"index":2,"previous_hash":"a3cdfeb5efdf64ac0844e10ff0ddf7fff53bb1283bf2bacaa187cb75a8805869","proof":35293,"timestamp":1640294690.6846914,"transactions":[{"amount":5,"recipient":"someone-other-address","sender":"d4ee26eee15148ee92c6cd394edd974e"},{"amount":1,"recipient":"3f0c3386a2684118ad9516a858bc8d6d5000","sender":"0"}]}],"message":"Our chain is authoritative"}
{"chain":[{"index":1,"previous_hash":1,"proof":100,"timestamp":1640294608.4698675,"transactions":[]},{"index":2,"previous_hash":"29a0086c3fc5334e1e58dc8b801ac9f01ccd998e900cb01a0faf05be2bf8dae0","proof":35293,"timestamp":1640294726.3293417,"transactions":[{"amount":1,"recipient":"a5afc77a149047c384f

In [ ]:
!curl http://localhost:5001/chain

{"chain":[{"index":1,"previous_hash":1,"proof":100,"timestamp":1640294608.4698675,"transactions":[]},{"index":2,"previous_hash":"29a0086c3fc5334e1e58dc8b801ac9f01ccd998e900cb01a0faf05be2bf8dae0","proof":35293,"timestamp":1640294726.3293417,"transactions":[{"amount":1,"recipient":"a5afc77a149047c384f899f2b84bfbb05001","sender":"0"}]}],"length":2}


In [ ]:
!curl http://localhost:5002/mine
!curl http://localhost:5000/nodes/resolve
!curl http://localhost:5001/nodes/resolve
!curl http://localhost:5002/nodes/resolve

{"index":3,"message":"New Block Forged","previous_hash":"5d20a12e29cf5dde954e036568393d90dcd278f2fd6f664ab60d200aef1c205b","proof":35089,"transactions":[{"amount":1,"recipient":"e38d3bd541244d3c9be583550af542c05002","sender":"0"}]}
{"message":"Our chain was replaced","new_chain":[{"index":1,"previous_hash":1,"proof":100,"timestamp":1640294608.456102,"transactions":[]},{"index":2,"previous_hash":"a3cdfeb5efdf64ac0844e10ff0ddf7fff53bb1283bf2bacaa187cb75a8805869","proof":35293,"timestamp":1640294690.6846914,"transactions":[{"amount":5,"recipient":"someone-other-address","sender":"d4ee26eee15148ee92c6cd394edd974e"},{"amount":1,"recipient":"3f0c3386a2684118ad9516a858bc8d6d5000","sender":"0"}]},{"index":3,"previous_hash":"5d20a12e29cf5dde954e036568393d90dcd278f2fd6f664ab60d200aef1c205b","proof":35089,"timestamp":1640294757.534341,"transactions":[{"amount":1,"recipient":"e38d3bd541244d3c9be583550af542c05002","sender":"0"}]}]}
{"message":"Our chain was replaced","new_chain":[{"index":1,"previo

In [ ]:
!curl http://localhost:5002/chain

{"chain":[{"index":1,"previous_hash":1,"proof":100,"timestamp":1640294608.456102,"transactions":[]},{"index":2,"previous_hash":"a3cdfeb5efdf64ac0844e10ff0ddf7fff53bb1283bf2bacaa187cb75a8805869","proof":35293,"timestamp":1640294690.6846914,"transactions":[{"amount":5,"recipient":"someone-other-address","sender":"d4ee26eee15148ee92c6cd394edd974e"},{"amount":1,"recipient":"3f0c3386a2684118ad9516a858bc8d6d5000","sender":"0"}]},{"index":3,"previous_hash":"5d20a12e29cf5dde954e036568393d90dcd278f2fd6f664ab60d200aef1c205b","proof":35089,"timestamp":1640294757.534341,"transactions":[{"amount":1,"recipient":"e38d3bd541244d3c9be583550af542c05002","sender":"0"}]}],"length":3}


In [ ]:
import signal
for cpid in out_pids:
    #os.waitpid(cpid, 0) # to wait for a peaceful exit
    os.kill(cpid, signal.SIGTERM)